### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [149]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
school_data.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [150]:
# Read Student Data File and store into Pandas DataFrames
student_data = pd.read_csv(student_data_to_load)
student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [151]:
# Combine the data into a single dataset.  
both_data = pd.merge(student_data, school_data, how="outer", on=["school_name"])
both_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [152]:
total_schools = both_data['school_name'].nunique()
total_students = student_data['Student ID'].count()
total_budget =  school_data['budget'].sum()

mean_math = student_data['math_score'].mean()
mean_reading = student_data['reading_score'].mean()

pass_math = [ x for x in student_data['math_score'] if x > 69]
percent_math_pass = len(pass_math) / len(student_data['math_score'])*100

pass_reading = [ x for x in student_data['reading_score'] if x > 69]
percent_reading_pass = len(pass_reading) / len(student_data['reading_score'])*100

#overall_pass_count = 0
both_pass = student_data[(student_data['reading_score'] > 69) & (student_data['math_score'] > 69)]
        
percent_all_pass = len(both_pass) / len(student_data)*100

#print(total_schools)
#print(total_students)
#print(total_budget)
#print(mean_math)
#print(mean_reading)
#print(percent_math_pass)
#print(percent_reading_pass)
#print(percent_all_pass)

district_summary = pd.DataFrame()

d =  {'Total Schools': total_schools , 
      'Total Students': total_students, 
      'Total Budget':total_budget, 
      'Average Math Score':mean_math,
      'Average Reading Score':mean_reading,
      '% Passing Math':percent_math_pass,
      '% Passing Reading':percent_reading_pass,
      '% Overall Passing':percent_all_pass}
df = pd.DataFrame([d], columns=d.keys())
print(df)
#df_to_csv('District_summary.csv')

   Total Schools  Total Students  Total Budget  Average Math Score  \
0             15           39170      24649428           78.985371   

   Average Reading Score  % Passing Math  % Passing Reading  % Overall Passing  
0               81.87784       74.980853          85.805463          65.172326  


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [159]:
#group by data
gp = both_data.groupby(['school_name','type'])

#find summary counts and totals per school/type
student_totals = gp['student_name'].count()
total_budget = gp['budget'].sum()
#merge both sets of data on index
mergedf = pd.merge(student_totals, total_budget, how='outer',left_index=True, right_index=True)
#add column budget per student
mergedf['Per Student Budget'] = mergedf['budget'] / mergedf['student_name']


#find mean reading and math scores per school/type
mean_reading_score = gp['reading_score'].mean()
mean_math_score = gp['math_score'].mean()
#merge both sets of data on index
mergedf2 = pd.merge(mean_math_score, mean_reading_score, how='outer',left_index=True, right_index=True)

#merge df1 and df 2 
mergedf3 = pd.merge(mergedf, mergedf2, how='outer',left_index=True, right_index=True)

#rename columns to clean up data
mergedf3.rename(columns={'student_name': 'Total Students',
                         'budget': 'Total School Budget',
                         'math_score': 'Average Reading Score',
                         'reading_score': 'Average Reading Score'}, inplace=True)
#mergedf3

#create new df only with passing scores for math, reading and both 

#
pass_math_df = both_data.loc[both_data['math_score'] > 69]
pass_reading_df = both_data.loc[both_data['reading_score'] > 69] 
pass_both_df = both_data.loc[(both_data['reading_score'] > 69) & (both_data['math_score'] > 69)]     

#group each passing score padata frame on school/type
gpmath = pass_math_df.groupby(['school_name','type'])
gpreading = pass_reading_df.groupby(['school_name','type'])
gpboth = pass_both_df.groupby(['school_name','type'])

#find the counts for each categrory 
totals_math_pass = gpmath['math_score'].count()
totals_reading_pass = gpreading['reading_score'].count()
totals_both_pass = gpboth['reading_score'].count()

#merge math and reading on index
mergeMR = pd.merge(totals_math_pass, totals_reading_pass, how='outer',left_index=True, right_index=True)
#merge with both on index
mergeBOTH = pd.merge(mergeMR, totals_both_pass, how='outer',left_index=True, right_index=True)
#rename columns
mergeBOTH.rename(columns={'math_score': 'Total Passing Math Students',
                         'reading_score_x': 'Total Passing Reading Students',
                         'reading_score_y': 'Total Passing Both Students'}, inplace=True)
#mergeBOTH

#merge all the data together
Merge_all = pd.merge(mergedf3, mergeBOTH, how='outer',left_index=True, right_index=True)
#Merge_all

#calculate percent passings in new columns 
Merge_all['% Passing Math'] = Merge_all['Total Passing Math Students'] / Merge_all['Total Students']*100
Merge_all['% Passing Reading'] = Merge_all['Total Passing Reading Students'] / Merge_all['Total Students']*100
Merge_all['% Overall Passing'] = Merge_all['Total Passing Both Students'] / Merge_all['Total Students']*100

del Merge_all['Total Passing Math Students']
del Merge_all['Total Passing Reading Students']
del Merge_all['Total Passing Both Students']

Merge_all

,,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,type,,,,,,,,
Bailey High School,District,4976,15549641728,3124928.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,2009159448,1081356.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,5557128039,1884411.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,4831365924,1763916.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,1346890000,917500.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,14007062700,3022020.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,105933149,248087.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,5573322295,1910635.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,14733628650,3094650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [161]:
Top_performers = Merge_all.sort_values('% Overall Passing')

print(Top_performers.head(5))


                                Total Students  Total School Budget  \
school_name           type                                            
Rodriguez High School District            3999          10186904637   
Figueroa High School  District            2949           5557128039   
Huang High School     District            2917           5573322295   
Hernandez High School District            4635          14007062700   
Johnson High School   District            4761          14733628650   

                                Per Student Budget  Average Reading Score  \
school_name           type                                                  
Rodriguez High School District           2547363.0              76.842711   
Figueroa High School  District           1884411.0              76.711767   
Huang High School     District           1910635.0              76.629414   
Hernandez High School District           3022020.0              77.289752   
Johnson High School   District          

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [163]:
Top_performers = Merge_all.sort_values('% Overall Passing', ascending=False)

print(Top_performers.head(5))

                             Total Students  Total School Budget  \
school_name         type                                           
Cabrera High School Charter            1858           2009159448   
Thomas High School  Charter            1635           1705517550   
Griffin High School Charter            1468           1346890000   
Wilson High School  Charter            2283           3012587442   
Pena High School    Charter             962            563595396   

                             Per Student Budget  Average Reading Score  \
school_name         type                                                 
Cabrera High School Charter           1081356.0              83.061895   
Thomas High School  Charter           1043130.0              83.418349   
Griffin High School Charter            917500.0              83.351499   
Wilson High School  Charter           1319574.0              83.274201   
Pena High School    Charter            585858.0              83.839917   

    

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [175]:
#student_data.head()
#create 4 data frames, one for each grade
grade9_df = student_data.loc[student_data['grade'] == '9th']
grade10_df = student_data.loc[student_data['grade'] == '10th']
grade11_df = student_data.loc[student_data['grade'] == '11th']
grade12_df = student_data.loc[student_data['grade'] == '12th']


#group data by school and then find average math scores
gp9 = grade9_df.groupby(['school_name'])
mean_math9 = gp9['math_score'].mean()

gp10 = grade10_df.groupby(['school_name'])
mean_math10 = gp10['math_score'].mean()

gp11 = grade11_df.groupby(['school_name'])
mean_math11 = gp11['math_score'].mean()

gp12 = grade12_df.groupby(['school_name'])
mean_math12 = gp12['math_score'].mean()

##Combine all summary data

#merge mean 9th and 10th on index
merged_9_10 = pd.merge(mean_math9, mean_math10, how='outer',left_index=True, right_index=True)
#remane columns 
merged_9_10.rename(columns={'math_score_x': '9th',
                            'math_score_y': '10th'}, inplace=True)
#merge mean 9th and 10th and 11th on index
merged_9_10_11 = pd.merge(merged_9_10, mean_math11, how='outer',left_index=True, right_index=True)

#merge mean 9th and 10th and 11th and 12th on index
merged_9_10_11_12 = pd.merge(merged_9_10_11, mean_math12, how='outer',left_index=True, right_index=True)
#remane columns 
merged_9_10_11_12.rename(columns={'math_score_x': '11th',
                                  'math_score_y': '12th'}, inplace=True)
merged_9_10_11_12


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [177]:
#student_data.head()
#create 4 data frames, one for each grade
grade9_df = student_data.loc[student_data['grade'] == '9th']
grade10_df = student_data.loc[student_data['grade'] == '10th']
grade11_df = student_data.loc[student_data['grade'] == '11th']
grade12_df = student_data.loc[student_data['grade'] == '12th']


#group data by school and then find average math scores
gp9 = grade9_df.groupby(['school_name'])
mean_reading9 = gp9['reading_score'].mean()

gp10 = grade10_df.groupby(['school_name'])
mean_reading10 = gp10['reading_score'].mean()

gp11 = grade11_df.groupby(['school_name'])
mean_reading11 = gp11['reading_score'].mean()

gp12 = grade12_df.groupby(['school_name'])
mean_reading12 = gp12['reading_score'].mean()

##Combine all summary data

#merge mean 9th and 10th on index
merged_9_10 = pd.merge(mean_reading9, mean_reading10, how='outer',left_index=True, right_index=True)
#remane columns 
merged_9_10.rename(columns={'reading_score_x': '9th',
                            'reading_score_y': '10th'}, inplace=True)
#merge mean 9th and 10th and 11th on index
merged_9_10_11 = pd.merge(merged_9_10, mean_reading11, how='outer',left_index=True, right_index=True)

#merge mean 9th and 10th and 11th and 12th on index
merged_9_10_11_12 = pd.merge(merged_9_10_11, mean_reading12, how='outer',left_index=True, right_index=True)
#remane columns 
merged_9_10_11_12.rename(columns={'reading_score_x': '11th',
                                  'reading_score_y': '12th'}, inplace=True)
merged_9_10_11_12

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [18]:
.cut and bin

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
